In [3]:
import pandas as pd
import numpy as np
import random 
import os

import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import log_loss, roc_curve, recall_score, precision_score, average_precision_score, f1_score, classification_report, accuracy_score,confusion_matrix, silhouette_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, cross_val_predict, train_test_split

import warnings
warnings.filterwarnings("ignore")
os.chdir("/Users/yj.noh/Documents/GitHub")
print(os.getcwd())

/Users/yj.noh/Documents/GitHub


In [5]:
data = pd.read_csv("/Users/yj.noh/Desktop/on_boarding_data.csv", encoding = "cp949")
print(data.head())
print(data.isna().sum())

                                       brms_rider_id  birth   delivery_method  \
0  7101DD2151A87A1BAAF00E96C2D74E05B86AC93F08AF7D...   1990      PAS_BICYCLES   
1  694AB03F4E692FFE0D11C50E09476161E1FAEB4C536BAE...   2000              WALK   
2  43BD4ABFA03CD6D25E3BE37CA98B1AFD1230F8DAB98E2A...   1970              BIKE   
3  CA68B1C8EB5B085F45A2093B1EAECB2BA7E8C73E729751...   1990               CAR   
4  5E41585113439D659B4D2A9B53CB75490C8CB8151B1F1E...   1980  GENERAL_BICYCLES   

  first_available_date business_start_day  is_recom gender  day_cnt  \
0           2023-07-22         2023-09-20         0    MAN      1.0   
1           2023-01-04         2023-01-05         1    MAN      3.0   
2           2023-01-09         2023-01-09         0    MAN      4.0   
3           2023-01-04                NaN         0    MAN      NaN   
4           2023-01-04         2023-01-04         1  WOMAN      1.0   

     avg_cnt last_working_day  outcome  
0   7.000000       2023-09-20        1  
1   

In [6]:
data_filter = data[data['day_cnt'].notnull()]
print(data_filter.isna().sum())

brms_rider_id           0
birth                   0
delivery_method         0
first_available_date    0
business_start_day      0
is_recom                0
gender                  0
day_cnt                 0
avg_cnt                 0
last_working_day        0
outcome                 0
dtype: int64


In [7]:
print(data_filter['outcome'].value_counts()) #0 : 30796, 1 : 21510 

outcome
1    30796
0    21510
Name: count, dtype: int64


In [11]:
X_train = data_filter[['avg_cnt', 'day_cnt']]
y_train = data_filter['outcome']

t(y_test, y_prob_pred, target_names=['normal', 'abnormal']))

model = LogisticRegression()
y_prob = model.predict()
y_scores = model.predict_proba(X_train)[:, 1]
fpr, tpr, thresholds = roc_curve(y_train, y_scores)

optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
optimal_X1_value = X_train[optimal_idx, 0]
optimal_X2_value = X_train[optimal_idx, 1]
print("최적의 threshold 값:", optimal_threshold)
print("입력 변수 1의 값:", optimal_X1_value)
print("입력 변수 2의 값:", optimal_X2_value)



NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.